### Problem 9.1 : Applying QSVM with PennyLane Circle Dataset

PennyLane is an open-source software framework for quantum machine learning, quantum chemistry, and quantum computing, with the ability to run on all hardware.

Link - https://pennylane.ai/datasets/


In [1]:
# !pip install pennylane

In [2]:
import pennylane as qml
import numpy as np

from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.kernels import FidelityQuantumKernel

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

In [3]:
def load_pennylane_data(n_samples=100, noise=0.1):
    """
    Load and prepare the PennyLane circle dataset
    """
    X, y = qml.datasets.circle(n_samples=n_samples, noise=noise)
    
    # Scale the features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    return X_scaled, y, X  # Return both scaled and unscaled data for visualization


In [4]:
def visualize_data(X, y, title="Circle Dataset"):
    """
    Visualize the 2D dataset
    """
    plt.figure(figsize=(10, 8))
    plt.scatter(X[y == 1][:, 0], X[y == 1][:, 1], c='b', marker='o', label='Class 1', alpha=0.6)
    plt.scatter(X[y == -1][:, 0], X[y == -1][:, 1], c='r', marker='x', label='Class -1', alpha=0.6)
    plt.title(title)
    plt.xlabel("Feature 1")
    plt.ylabel("Feature 2")
    plt.legend()
    plt.grid(True)
    plt.show()

In [5]:
def create_and_train_qsvm(X_train, X_test, y_train, y_test):
    """
    Create and train the QSVM model
    """
    # Create quantum kernel
    feature_map = ZZFeatureMap(
        feature_dimension=2,  # Circle dataset has 2 features
        reps=3,              # Increased reps for better performance
        entanglement='full'  # Full entanglement for better feature mapping
    )
    
    quantum_kernel = FidelityQuantumKernel(
        feature_map=feature_map
    )
    
    # Compute kernel matrices
    print("Computing kernel matrices...")
    kernel_matrix_train = quantum_kernel.evaluate(X_train, X_train)
    kernel_matrix_test = quantum_kernel.evaluate(X_test, X_train)
    
    # Train SVM
    from sklearn.svm import SVC
    svm = SVC(kernel='precomputed')
    svm.fit(kernel_matrix_train, y_train)
    
    # Make predictions
    y_pred = svm.predict(kernel_matrix_test)
    
    return y_pred

In [6]:
def plot_decision_regions(X, y, model, quantum_kernel, title="Decision Regions"):
    """
    Plot decision regions for the model
    """
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    
    # Create a mesh grid
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02),
                        np.arange(y_min, y_max, 0.02))
    
    # Create mesh points
    mesh_points = np.c_[xx.ravel(), yy.ravel()]
    
    # Compute kernel between mesh points and training data
    kernel_matrix = quantum_kernel.evaluate(mesh_points, X)
    
    # Make predictions
    Z = model.predict(kernel_matrix)
    Z = Z.reshape(xx.shape)
    
    # Plot
    plt.figure(figsize=(10, 8))
    plt.contourf(xx, yy, Z, alpha=0.4)
    plt.scatter(X[y == 1][:, 0], X[y == 1][:, 1], c='b', marker='o', label='Class 1')
    plt.scatter(X[y == -1][:, 0], X[y == -1][:, 1], c='r', marker='x', label='Class -1')
    plt.title(title)
    plt.xlabel("Feature 1")
    plt.ylabel("Feature 2")
    plt.legend()
    plt.show()


In [7]:
# Load and prepare data
print("Loading PennyLane circle dataset...")
X_scaled, y, X_original = load_pennylane_data(n_samples=100, noise=0.1)

# Visualize original data
print("\nVisualizing original dataset...")
visualize_data(X_original, y, "Original Circle Dataset")

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42
)

# Train and evaluate QSVM
print("\nTraining QSVM...")
y_pred = create_and_train_qsvm(X_train, X_test, y_train, y_test)

# Print results
print("\nResults:")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Loading PennyLane circle dataset...


AttributeError: module 'pennylane' has no attribute 'datasets'